# HW7: Beam Search Decoding - News Headline Generation

In this exercise, you are going to learn and implement decoding techniques for sequence generation. Usually, the sequence is generated word-by-word from a model. In each step, the model predicted the most likely word based on the predicted words in previous steps (this is called auto-regressive decoding).

As such, it is very important how you decide on what to predicted at each step, as it will be conditioned on to predicted all of the following steps. We will implement two of main decoding techniques introduced in the lecture: **Greedy Decoding** and **Beam Search Decoding**. Greedy Decoding immediately chooses the word with best score at each step, while Beam Search Decoding focuses on the sequence that give the best score overall.

To complete this exercise, you will need to complete the methods for decoding for a text generation model trained on [New York Times Comments and Headlines dataset](https://www.kaggle.com/aashita/nyt-comments). The model is trained to predict a headline for the news given seed text. You do not need to train any model model in this exercise as we provide both the pretrained model and dictionary.


## Download model and vocab and setup

In [1]:
# !wget -O vocab.txt "https://www.dropbox.com/s/ht12ua9vpkep6l8/hw9_vocab.txt?dl=0"
# !wget -O model.bin "https://www.dropbox.com/s/okmri7cnd729rr5/hw9_model.bin?dl=0"

In [2]:
import torch
import torch.nn as nn
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace

In [3]:
class RNNmodel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout_rate):

        super().__init__()
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, 128, num_layers=2,
                     batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, vocab_size)

    def forward(self, src):
        embedding = self.embedding(src)
        output,_ = self.rnn(embedding)
        output = self.dropout(output)
        prediction = self.fc2(output)
        return prediction

In [4]:
with open("vocab.txt") as f:
  vocab_file = f.readlines()
embedding_dim = 64
dropout_rate = 0.2

model = RNNmodel(len(vocab_file), embedding_dim, dropout_rate)
model.load_state_dict(torch.load("model.bin",map_location='cpu'))
model.eval()

RNNmodel(
  (embedding): Embedding(10054, 64)
  (rnn): LSTM(64, 128, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=10054, bias=True)
)

In [5]:
vocab = [v.strip() for v in vocab_file]
vocab_size = len(vocab)
print(f"Vocab Size: {vocab_size}")
vocab[:10]

Vocab Size: 10054


['<unk>', '<pad>', '<eos>', 'the', 'a', 'to', 'of', 's', 'in', 'for']

In [6]:
stoi = { ch:i for i,ch in enumerate(vocab) }
tokenizer = Tokenizer(WordLevel(stoi, unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
tokenized_text = tokenizer.encode("the a of to unknowns")
print(tokenized_text)
print(tokenized_text.ids)
print(tokenized_text.tokens)
print(tokenizer.decode(tokenized_text.ids))

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[3, 4, 6, 5, 0]
['the', 'a', 'of', 'to', '<unk>']
the a of to <unk>


## 1. TODO: Greedy decode
Normally, in sequence generation task, the model will continue generating tokens until an end-of-sequence symbol appear or the maximum length is reached. For this task:
- The end-of-sequence symbol is "< eos >" and its index is 2
- Use the maximum generation length of 15

In [7]:
# TODO:
from typing import List, Tuple

eos_token = "<eos>"
eos_id = vocab.index(eos_token)
max_gen_length = 15


def greedy_decode(
    seed_text: str, tokenizer, max_gen_length=15
) -> Tuple[List[int], List[float]]:
    """Greedy decodes with seed text.

    Args:
    seed_text: The seed string to be used as initial input to the model.
    tokenizer: The tokenizer for converting word to index and back.

    Your code should do the followings:
      1. Convert current_text to sequences of indices
      2. Predict the next token using the model and choose the token with the highest score as output
      3. Append the predicted index to current_text
      4. Loop until completion
      5. Return text prediction and a list of probabilities of each step

    You do not need to stop early when end-of-sequence token is generated and can continue decoding
    until max_gen_length is reached. We can filter the eos token out later.
    """
    probs: List[float] = []
    # Convert the seed text into token IDs and create a tensor
    current_text = torch.Tensor(tokenizer.encode(seed_text).ids).unsqueeze(0).long()
    is_eos = False # Flag to indicate if the end-of-sequence token is reached

    # Continue decoding until the maximum generation length is reached
    while len(current_text[0]) < max_gen_length:
        # Predict
        prediction = model(current_text)
        pred_probs = nn.functional.softmax(prediction[0], dim=-1)

        # Top
        top_probs, top_token_ids = torch.topk(pred_probs, 1)
        top_probs.squeeze_(1)
        top_token_ids.squeeze_(1)

        # Check if the predicted token is the end-of-sequence token
        if top_token_ids[-1].item() == eos_id:
            is_eos = True
        if not is_eos:
            probs.append(top_probs[-1].item())
        # Append the predicted token ID to the current text
        current_text = torch.cat(
            (current_text, torch.tensor([[top_token_ids[-1]]])), dim=1
        )
    # Return the complete sequence of token IDs and the corresponding list of probabilities
    output = current_text[0].tolist()
    return output, probs

In [8]:
def clean_output(token_ids, eos_token_id) -> str:
    """Drop eos_token and every words that follow"""
    # TODO:
    if eos_token_id in token_ids:
        eos_index = token_ids.index(eos_token_id)
        token_ids = token_ids[:eos_index]
    return tokenizer.decode(token_ids)

In [9]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
for seed in sample_seeds:
    # TODO:
    output_text, probs = greedy_decode(seed, tokenizer)
    cleaned_text = clean_output(output_text, stoi[eos_token])
    print(f"Seed: {seed}\nOutput: {cleaned_text}\nProbabilities: {probs}\n")

Seed: to
Output: to encourage creativity in the new york bill
Probabilities: [0.04071057215332985, 0.26613089442253113, 0.258160263299942, 0.2311653345823288, 0.07794327288866043, 0.06978043168783188, 0.224452406167984]

Seed: america
Output: america s lethal export
Probabilities: [0.44965630769729614, 0.046399760991334915, 0.7206984758377075]

Seed: people
Output: people to balloon to make a criminal with a dog with a callous rival
Probabilities: [0.35848551988601685, 0.039465878158807755, 0.2695296108722687, 0.03377290070056915, 0.28156769275665283, 0.030896661803126335, 0.32625600695610046, 0.3224417567253113, 0.03489115834236145, 0.22298383712768555, 0.5808295607566833, 0.04674490541219711, 0.26616615056991577]

Seed: next
Output: next phenom english clubs 2 call another deal in the same arrivals
Probabilities: [0.17242398858070374, 0.2323562502861023, 0.7396275997161865, 0.07888377457857132, 0.04404321685433388, 0.05769364535808563, 0.03296956792473793, 0.33040109276771545, 0.6040

Your output should be:

*   to encourage creativity in the new york bill
*   america s lethal export
*   people to balloon to make a criminal with a dog with a callous rival
*   next phenom english clubs 2 call another deal in the same arrivals
*   picture perfect chapter a spot of view of banning care  
*   on the catwalk in saudi arabia







## 2. TODO: Beam search decode

Another well-known decoding method is beam search decoding that focuses more on the overall sequence score.

Instead of greedily choosing the token with the highest score for each step, beam search decoding expands all possible next tokens and keeps the __k__ most likely sequence at each step, where __k__ is a user-specified beam size. A sequence score is also calculated according user-specified cal_score() function.
The beam with the highest score after the decoding process is done will be the output.

There are a few things that you need to know before implementing a beam search decoder:
- When the eos token is produced, you can stop expanding that beam
- However, the ended beams must be sorted together with active beams
- The decoding ends when every beams are either ended or reached the maximum length, but for this task, you can continue decoding until the max_gen_len is reached
- We usually work with probability in log scale to avoid numerical underflow. You should use np.log(score) before any calculation
- **As probabilities for some classes will be very small, you must add a very small value to the score before taking log e.g np.log(prob + 0.00000001)**

#### Sequence Score
The naive way to calculate the sequence score is to __multiply every token scores__ together. However, doing so will make the decoder prefer shorter sequence as you multiply the sequence score with a value between \[0,1\] for every tokens in the sequence. Thus, we usually normalize the sequence score with its length by calculating its __geometric mean__ instead.

**You should do this in log scale**

In [10]:
import numpy as np

def cal_score(score_list, length, normalized=False) -> float:
    # TODO: cal score for each beam from a list of probs
    log_scores = [np.log(s + 1e-15) for s in score_list]
    if normalized:
        return np.sum(log_scores) / length
    else:
        return np.sum(log_scores)

In [11]:
def beam_search_decode(seed_text, max_gen_len, tokenizer, beam_size=5, normalized=False):
    """We will do beam search decoding using seed text in this function.

    Output:
    beams: A list of top k beams after the decoding ended, each beam is a list of
      [seed_text, list of scores, length]

    Your code should do the followings:
    1. Loop until max_gen_len is reached.
    2. During each step, loop through each beam and use it to predict the next word.
       If a beam is already ended, continues without expanding.
    3. Sort all hypotheses according to cal_score().
    4. Keep top k hypotheses to be used at the next step.
    """
    # TODO:
    # Convert the initial seed text into token IDs and create the initial beam with it.
    # Each beam consists of (token_ids, list_of_scores, length, is_finished).
    seed_ids = torch.Tensor(tokenizer.encode(seed_text).ids).unsqueeze(0).long()
    beams = [(seed_ids, [], 0, False)]  # Initialize with the seed text

    # Iterate over the maximum generation length
    for _ in range(max_gen_len):
        new_beams = []
        # For each beam, predict the next token unless the beam has ended
        for beam in beams:
            if beam[-1]:  # If is_finished == True, skip expansion
                new_beams.append(beam)
                continue

            token_ids, score_list, length, _ = beam
            prediction = model(token_ids)  # Get model predictions
            pred_probs = nn.functional.softmax(prediction[0], dim=-1)  # Convert logits to probabilities
            next_token_pred_probs = pred_probs[-1]  # Only consider the last token probabilities

            # Expand each beam by adding each possible next token
            for next_token_id, next_token_prob in enumerate(next_token_pred_probs):
                new_text = torch.cat((token_ids, torch.tensor([[next_token_id]])), dim=1)
                new_length = length + 1
                new_finished = (next_token_id == stoi[eos_token])  # Check if eos token reached
                new_score = score_list + [next_token_prob.item()]  # Append new probability score
                new_beams.append((new_text, new_score, new_length, new_finished))

        # Sort the new beams based on calculated scores (normalized or not) and keep top k
        new_beams.sort(key=lambda x: cal_score(x[1], x[2], normalized), reverse=True)
        beams = new_beams[:beam_size]  # Retain top k beams for next iteration

    # Return the final list of beams after max_gen_len iterations
    return beams

## 3. Generate!
Generate 6 sentences based on the given seed texts.

Decode with the provided seed texts with beam_size 5, max_gen_len 10. Compare the results between greedy, normalized, and unnormalized decoding.

Print the result using greedy decoding and top 2 results each using unnormalized and normalized decoing for each seed text.

Also, print scores of each candidate according to cal_score(). Use normalization for greedy decoding.

In [12]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
max_gen_length = 10

def generate(sample_seeds):
    for seed in sample_seeds:
        def _greedy():
            print("-Greedy-")
            output_text, probs = greedy_decode(seed, tokenizer)
            cleaned_text = clean_output(output_text, stoi[eos_token])
            score = cal_score(probs, len(probs), normalized=True)
            print(f"{cleaned_text}  {round(np.exp(score), 2)}")
        _greedy()

        def _unnormalized():
            print("-Unnormalized-")
            beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=False)
            for beam in beams[:2]:
                output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
                if beam[3]:
                    score = cal_score(beam[1][:-1], beam[2]-1, normalized=False)
                else:
                    score = cal_score(beam[1], beam[2], normalized=False)
                print(f"{output_text.title()}  {round(np.exp(score), 2)}")
        _unnormalized()

        def _normalized():
            print("-Normalized-")
            beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=True)
            for beam in beams[:2]:
                output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
                if beam[3]:
                    score = cal_score(beam[1][:-1], beam[2]-1, normalized=True)
                else:
                    score = cal_score(beam[1], beam[2], normalized=True)
                print(f"{output_text.title()}  {round(np.exp(score), 2)}")
        _normalized()

        print() # newline

generate(sample_seeds)

-Greedy-
to encourage creativity in the new york bill  0.13
-Unnormalized-
To Consult Exploring Recipes For New Jersey  0.0
To Consult Exploring Recipes Up The Pacific Northwest  0.0
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest  0.17
To Consult Exploring Recipes Up The Least Of The Week  0.16

-Greedy-
america s lethal export  0.25
-Unnormalized-
America S Lethal Export  0.02
America S Desert Aisles  0.01
-Normalized-
America S Lethal Export  0.25
America S Desert Aisles  0.2

-Greedy-
people to balloon to make a criminal with a dog with a callous rival  0.14
-Unnormalized-
People To Balloon For A Criminal  0.0
People To Balloon For A Criminal With Trump  0.0
-Normalized-
People To Balloon For A Criminal With A Second Fiddle  0.13
People To Balloon For A Criminal With Trump  0.13

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals  0.12
-Unnormalized-
Next S Blist Revue  0.0
Next Phenom English Clubs 1 A Chance To Be Back  0.0
-Normalized-

Your output should be:


```
-Greedy-
to encourage creativity in the new york bill  0.13
-Unnormalized-
To Consult Exploring Recipes For New Jersey 0.00
To Consult Exploring Recipes Up The Pacific Northwest 0.00
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest 0.17
To Consult Exploring Recipes Up The Least Of The Week 0.16

-Greedy-
america s lethal export  0.25
-Unnormalized-
America S Lethal Export 0.02
America S Desert Aisles 0.01
-Normalized-
America S Lethal Export 0.25
America S Desert Aisles 0.20

-Greedy-
people to balloon to make a criminal with a dog with a callous rival  0.14
-Unnormalized-
People To Balloon For A Criminal 0.00
People To Balloon For A Criminal With Trump 0.00
-Normalized-
People To Balloon For A Criminal With A Second Fiddle 0.13
People To Balloon For A Criminal With Trump 0.13

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals  0.12
-Unnormalized-
Next S Blist Revue 0.00
Next Phenom English Clubs 1 A Chance To Be Back 0.00
-Normalized-
Next S Blist Revue 0.14
Next Phenom English Clubs 1 A Chance To Be Back 0.14

-Greedy-
picture perfect chapter a spot of view of banning care  0.08
-Unnormalized-
Picture Perfect Use Coffee 0.00
Picture Korean A Bonanza Of Pancakes 0.00
-Normalized-
Picture Korean A Bonanza Of Contemplation Times Of Trump S Son 0.12
Picture Korean A Bonanza Of Pancakes 0.07

-Greedy-
on the catwalk in saudi arabia  0.19
-Unnormalized-
On The Billboard Chart 0.00
On The Catwalk In Saudi Arabia 0.00
-Normalized-
On The Whole30 Diet Vowing To Eat Smarter Carbs To Be 0.27
On The Whole30 Diet Vowing To Eat Smarter Carbs For Because 0.26

```



# Answer Questions in MyCourseVille!

Use the seed word "usa" to answer questions in MCV.

In [13]:
generate(["usa"])

-Greedy-
usa s duty to investigate  0.09
-Unnormalized-
Usa S Duty To Investigate  0.0
Usa S Bleak Season 3 Episode 4 Recap Chicken  0.0
-Normalized-
Usa S Bleak Season 1 Episode 2 Darkness Descends  0.19
Usa S Bleak Season 3 Episode 4 Recap A Seduction  0.18

